## Convert Classic Esri Story Map Series

**Welcome!**  
This notebook will guide you through converting a Classic Esri Story Map Series to an ArcGIS StoryMap Collection.

**How to run this notebook**  
 - Click on the text "Setup and authenticate" below. 
 - There are two types of cells, Markdown (formatted notes) and Code.
 - An indicator -- typically a vertical blue line -- should highlight that you have selected the Markdown cell. 
 - Once selected, click the "Play" button in the toolbar above to run the cell and advance to the next Code cell.
 - Click the "Play" button a second time to run the code cell.
 - After several seconds a "Setup Notebook" button should appear. Click the button to begin setup and authentication.
 - After each cell completes, click the text within the following Markdown cell.
 - Click the "Play" button to advance to the Code cell, then click the "Play" button a second time to make a button appear.
 - Click the button to run the code in the cell. 

 **Notes**  
 - Some code cells may take a while to execute. You can monitor the status by viewing the small circle in the top right of the page.
 - If you click on a code cell it will expand showing you the behind-the-scenes Python code. 
 - For a cleaner interface select View > Collapse All Code in the menu bar above to hide the code .
 - If at any point you get stuck and want to start over, just click Kernel > Restart Kernel and Clear Outputs of All Cells... in the menu bar 

**tldr;**

In [ ]:
# Run this cell to display Notebook details
from IPython.display import display, Markdown

# Display details of what this notebook does
tldr_md = """
**What this notebook does**  
Fetch JSON from an ArcGIS Online hosted Classic Esri Story Map Series App and convert each tab/bullet/accordion into its own ArcGIS StoryMap with the cover supressed. Once converted, each ArcGIS StoryMap will need to be opened in a browser tab in order to complete the Story Checker. Once all are published, an ArcGIS StoryMap Collection is created that contains the converted app to replicate the classic app look and feel. Note: Any entries that were hidden in the classic app will be published and will be visible by default. If it is desired that they not appear they can be removed from the Collection after publishing. Also, as there is not equivalent to the accordion layout, these layouts will be converted to the Tabbed format.
"""
display(Markdown(tldr_md))

TO DO - create conversion tool for classic swipe (second tab in Katrina story)

## 1. Setup and authenticate

In [ ]:
# In the toolbar above, select View > Collapse All Code to hide the code.
print("Initializing...")
# Cell 1. Import packages, config, AGO authentication and helper functions
import ipywidgets as widgets
from bs4 import BeautifulSoup, NavigableString, Tag
from arcgis.apps.storymap import StoryMap, Themes, Image, Video, Audio, Embed, Map, Button, Text, Gallery, Timeline, Sidecar, Code, Table, TextStyles, Collection, CollectionNavigation
import arcgis
from arcgis.gis import GIS, Item
from arcgis.map import Map as AGOMap
from PIL import Image as PILImage
from PIL import ImageStat
from io import BytesIO
from IPython.display import display, Markdown
import pandas as pd
import matplotlib.colors as mcolors
import tempfile
import ipywidgets as widgets
from ipywidgets import IntProgress
import re, json, requests, sys, time, os, subprocess
from copy import deepcopy
import warnings
from bs4 import MarkupResemblesLocatorWarning

# Suppress the BeautifulSoup warning when ArcGIS Notebooks thinks HTML looks like a locator/filename
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Set Pandas dataframe display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',1000)

# Declare global gis variable
gis = None
output1 = widgets.Output()
def setup_notebook(button):
    with output1:
        print("Setting up the notebook environment...")
        # Print Python and ArcGIS for Python versions
        # since things can change between versions
        print(f"\tPython version: ",sys.version)
        print(f"\tArcGIS for Python API / StoryMap module version: ",arcgis.__version__)
        # Connect to ArcGIS Online
        # Define the GIS
        agoNotebook = input_param1.value
        global gis
        if agoNotebook == False:
            username_widget = widgets.Text(
                value='',
                placeholder='Enter your ArcGIS Online username',
                #description='Username:',
                layout=widgets.Layout(width='400px')
            )
            submit_button = widgets.Button(description="Login")
            auth_Hbox = widgets.HBox([username_widget, submit_button])
            output_auth = widgets.Output()
            display(auth_Hbox, output_auth)

            def handle_login(b):
                with output_auth:
                    output_auth.clear_output()
                    print("Logging in...")
                    try:
                        import keyring
                        service_name = "system"
                        username_for_keyring = username_widget.value
                        credential = keyring.get_credential(service_name, username_for_keyring)
                        if credential is None:
                            print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
                        else:
                            password_from_keyring = keyring.get_password("system", username_for_keyring)
                            portal_url = 'https://www.arcgis.com'
                            global gis
                            gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
                            print(f"\tSuccessfully logged in as: {gis.properties.user.username} (role: {gis.properties.user.role} / userType: {gis.properties.user.userLicenseTypeId})")
                            print("\nStep #1 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
                    except ImportError:
                        print("The 'keyring' module is not installed. Please install it using 'pip install keyring'.")
                        print("Before re-running this cell, open a command line window on your machine and run the command:")
                        print("# python -m keyring set system <your_ago_username>")
                        print("If using Windows Powershell, use:")
                        print("# ./python -m keyring set system <your_ago_username>")
                        print("You will be prompted to enter your password")
                        print("When you hit Enter/Return the password will be saved to your local credential store.")

            submit_button.on_click(handle_login)
        else:
            gis = GIS("home")
            print(f"\tSuccessfully logged in as: {gis.properties.user.username} (role: {gis.properties.user.role} / userType: {gis.properties.user.userLicenseTypeId})")
            print("\nStep #1 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
        
#########################################################    
## Helper functions

default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"

# Collect map extents for troubleshooting thumbnail generation
map_extents = []
extent_records = []

# def normalize_folder_name(name):
#     # Remove leading/trailing whitespace 
#     name = name.strip()
#     # Replace underscores
#     name = name.replace('_', ' ')
#     # Lowercase string
#     # name = name.lower()
#     return name

def normalize_classic_name(classic_string):
    # Remove leading/trailing whitespace 
    classic_string = classic_string.strip()
    return classic_string

# def ensure_folder(gis, classic_title):
#     """
#     Checks if a folder with the specified name exists in the user's content, and creates it if not.
#     """
#     user = gis.users.me
#     existing_folders = gis.content.folders.list(user.username)
#     folder_names = [f.name for f in existing_folders]
#     # normalized_existing = [normalize_folder_name(f.name) for f in existing_folders]
#     # normalized_target = normalize_folder_name(folder_string)
#     if classic_title in folder_names:
#         print(f"Folder '{classic_title}' already exists. Saving results there.")
#     else:
#         try:
#             gis.content.folders.create(folder=classic_title, owner=user.username)
#             print(f"Created folder '{classic_title}' to save entries and Collection.")
#         except Exception as e:
#             print(f"Error creating folder '{classic_title}': {e}")
#     return classic_title

def check_folder(button):
    global classic_story_title, folder_name, output7, input_param7, user_button7_1
    with output7:
        output7.clear_output()
        if not classic_story_title:
            print("No classic StoryMap title found. Extract the story settings first.")
            return
        folder_name = "Collection-" + classic_story_title if classic_story_title else "Collection-Conversion"
        input_param7.value = folder_name
        user_line7 = widgets.HBox([widgets.Label(value="Edit the folder name if desired -->"), input_param7])
        # Check if folder exists
        user = gis.users.me
        existing_folders = gis.content.folders.list(user.username)
        folder_names = [f.name for f in existing_folders]
        if folder_name in folder_names:
            print(f"Folder '{folder_name}' already exists. Saving results there.")
            print("\nStep #7 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
        else:
            display(user_line7)
            display(user_button7_1)

def create_folder(button):
    global folder_name, output7, input_param7
    with output7:
        output7.clear_output()
        folder_name = input_param7.value.strip() if input_param7.value.strip() else folder_name
        try:
            gis.content.folders.create(folder=folder_name, owner=gis.users.me.username)
            print(f"Created folder '{folder_name}' to save entries and Collection.")
        except Exception as e:
            print(f"Error creating folder '{folder_name}': {e}")
        print("\nStep #7 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")


def fetch_classic_storymap_data(classic_storymap_id, gis):
    classic_item = Item(gis=gis, itemid=classic_storymap_id)
    classic_data = Item.get_data(classic_item)
    if classic_data == {}:
        raise ValueError("ERROR: StoryMap to be converted must be hosted on ArcGIS Online.")
    elif isinstance(classic_data, dict):
        classic_item_data = classic_data
    else:
        classic_item_data = json.loads(classic_data)
    return classic_item, classic_item_data

def fetch_classic_storymap_data(classic_storymap_id, gis):
    classic_item = Item(gis=gis, itemid=classic_storymap_id)
    classic_data = Item.get_data(classic_item)
    if classic_data == {}:
        raise ValueError("ERROR: StoryMap to be converted must be hosted on ArcGIS Online.")
    elif isinstance(classic_data, dict):
        classic_item_data = classic_data
    else:
        classic_item_data = json.loads(classic_data)
    return classic_item, classic_item_data

def extract_story_settings(classic_item_data):
    title = classic_item_data["values"].get("title", "Untitled Classic StoryMap Series")
    title = normalize_classic_name(title)
    subtitle = classic_item_data["values"].get("subtitle", "")

    if "values" in classic_item_data and "settings" in classic_item_data["values"]:
        settings = classic_item_data["values"]["settings"]
        story_type = settings["layout"]["id"]
        panel_position = settings["layoutOptions"]["panel"]["position"]
        theme = settings["theme"]
    else:
        settings = {}
        story_type = "Unknown or unsupported classic story"
        panel_position = "Unknown"
        theme = {}
        print(f"WARNING: Unsupported story type detected!")

    if "webmap" in classic_item_data["values"]:
        webmap_ids = classic_item_data["values"]["webmap"]
        print(f"\nThese webmaps are included in the story: {webmap_ids}. \nYou may be able to use them to recreate the Classic StoryMap using the ArcGIS Collection builder.")

    if "story" in classic_item_data["values"] and "entries" in classic_item_data["values"]["story"]:
        entries = classic_item_data["values"]["story"]["entries"]
    else:
        entries = []
    return title, subtitle, story_type, panel_position, theme, entries

def determine_theme(theme):
    classic_name = theme["colors"].get("name", "No classic theme name")
    group = theme["colors"]["group"]
    if group == "dark":
        return classic_name, Themes.OBSIDIAN
    elif group == "light":
        return classic_name, Themes.SUMMIT
    else:
        return classic_name, Themes.SUMMIT

def process_entry(entry):
    entry_title = entry.get("title")
    media_info = entry.get("media", {})
    media_type = media_info.get("type")
    main_stage_content = None
    invalid_webmap = False

    if media_type == "webmap":
        webmap_id = media_info.get('webmap', {}).get('id')
        map_extents.append((entry_title, webmap_id, media_info.get('webmap', {}).get('extent')))
        if webmap_id and not invalid_webmap:
            try:
                main_stage_content = Map(webmap_id)
            except Exception as e:
                print(f"Error creating Map object for {entry_title} ({webmap_id}): {e}")
                invalid_webmap = True
    elif media_type == "webpage":
        webpage_url = media_info.get("webpage", {}).get("url")
        if webpage_url:
            main_stage_content = Embed(webpage_url)
    elif media_type == "image":
        image_url = media_info.get("image", {}).get("url")
        if image_url:
            main_stage_content = Image(image_url)

    return entry_title, main_stage_content, invalid_webmap

def normalize_webmercator_x(x):
    """
    Wraps longitude values (meters) into the valid Web Mercator range for the ArcGIS print service.
    """
    min_x = -20037508.342789244
    max_x =  20037508.342789244
    world_width = max_x - min_x
    while x < min_x:
        x += world_width
    while x > max_x:
        x -= world_width
    return x

def normalize_extent_to_webmercator(extent):
    """
    Normalizes an extent dict to valid Web Mercator bounds by wrapping x values.
    """
    min_x = -20037508.342789244
    max_x =  20037508.342789244
    # Wrap x values
    xmin = normalize_webmercator_x(extent['xmin'])
    xmax = normalize_webmercator_x(extent['xmax'])
    # Clamp y values (no wrapping for y)
    ymin = max(min(extent['ymin'], max_x), min_x)
    ymax = max(min(extent['ymax'], max_x), min_x)
    return {
        'xmin': xmin,
        'ymin': ymin,
        'xmax': xmax,
        'ymax': ymax,
        'spatialReference': extent.get('spatialReference', {'wkid': 102100})
    }

def build_webmap_from_json(gis, media_info):
    """
    Build a minimal webmap JSON for the print service from a storymap entry's media property,
    using the basemap from the referenced webmap item if available.
    """
    # Default basemap (fallback)
    topo_basemap = {
        "baseMapLayers": [{
            "id": "World_Topo_Map",
            "layerType": "ArcGISTiledMapServiceLayer",
            "opacity": 1,
            "visibility": True,
            "url": "https://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer"
        }],
        "title": "Topographic"
    }

    imagery_basemap = {
        "baseMapLayers": [{
            "id": "World_Imagery",
            "layerType": "ArcGISTiledMapServiceLayer",
            "opacity": 1,
            "visibility": True,
            "url": "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer"
        }],
        "title": "Imagery"
    }
    baseMap = topo_basemap # Default if basemap not present

   # Get extent and spatial reference from the referenced webmap
    extent = None
    spatialRef = {"wkid": 102100}
    webmap = media_info.get("webmap", {})

    # --- Basemap ---
    if "baseMap" in webmap:
        # Use the baseMap as-is from the entry
        baseMap = deepcopy(webmap["baseMap"])
    else:
        # Try to get basemap from the referenced webmap item
        if "id" in media_info["webmap"]:
            try:
                webmap_item = gis.content.get(media_info["webmap"]["id"])
                webmap_item_data = webmap_item.get_data()
                if "baseMap" in webmap_item_data and "baseMapLayers" in webmap_item_data["baseMap"]:
                    # Only keep required fields for each basemap layer
                    baseMapLayers = []
                    for lyr in webmap_item_data["baseMap"]["baseMapLayers"]:
                        baseMapLayers.append({
                            "id": lyr.get("id", "basemap"),
                            "layerType": lyr.get("layerType", "ArcGISTiledMapServiceLayer"),
                            "opacity": lyr.get("opacity", 1),
                            "visibility": lyr.get("visibility", True),
                            "url": lyr.get("url")
                        })
                    baseMap = {
                        "baseMapLayers": baseMapLayers,
                        "title": webmap_item_data["baseMap"].get("title", "Basemap")
                    }
            
            except Exception as e:
                print(f"Could not fetch basemap from webmap item: {e}. Using fallback basemap.")

    # Normalize webmap extent if present
    if "extent" in webmap and webmap["extent"]:
        webmap["extent"] = normalize_extent_to_webmercator(webmap["extent"])
        spatialRef = webmap["extent"].get("spatialReference", spatialRef)
    
    webmap_extent = webmap.get("extent")
    if webmap_extent:
        extent = webmap_extent
    # Default to globe if extent not available
    else:
        extent = {
            "xmin": -20037508.342789244,
            "ymin": -20037508.342789244,
            "xmax": 20037508.342789244,
            "ymax": 20037508.342789244,
            "spatialReference": spatialRef
        }
    mapOptions = {"extent": extent}

    # --- Operational Layers ---
    if "operationalLayers" in webmap:
        # Use operationalLayers as-is from the entry
        operationalLayers = deepcopy(webmap["operationalLayers"])
    elif "layers" in webmap:
        # Fallback 
        operationalLayers = []
        if "webmap" in media_info and "layers" in media_info["webmap"]:
            layers = media_info["webmap"]["layers"]
            if layers:
                for lyr in layers:
                    # Try to get URL from referenced webmap item
                    layer_url = lyr.get("url")
                    if not layer_url and "id" in lyr and "id" in media_info["webmap"]:
                        try:
                            webmap_item = gis.content.get(media_info["webmap"]["id"])
                            webmap_item_data = webmap_item.get_data()
                            for op_lyr in webmap_item_data.get("operationalLayers", []):
                                if op_lyr.get("id") == lyr["id"]:
                                    layer_url = op_lyr.get("url")
                                    break
                        except Exception as e:
                            print(f"Could not fetch webmap item for layer URL lookup: {e}")
                    if layer_url:
                        operationalLayers.append({
                            "id": lyr.get("id", "layer"),
                            "layerType": lyr.get("layerType", "ArcGISFeatureLayer"),
                            "url": layer_url,
                            "visibility": lyr.get("visibility", True),
                            "opacity": lyr.get("opacity", 1)
                        })
    else:
        operationalLayers = []                        

    # Export options for print service
    export_options = {"outputSize": [800, 600], "dpi": 96}

    webmap_json = {
        "baseMap": baseMap,
        "operationalLayers": operationalLayers,
        "spatialReference": spatialRef,
        "mapOptions": mapOptions,
        "exportOptions": export_options   
    }    
    
    # Invert the drawing order of operational layers
    if "operationalLayers" in webmap_json:
        webmap_json["operationalLayers"] = list(reversed(webmap_json["operationalLayers"]))

    return webmap['id'], webmap_json

def create_image_thumbnail(image_url, default_thumbnail_path):
    try:
        response = requests.get(image_url)
        img = PILImage.open(BytesIO(response.content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.thumbnail((800, 600))
        img.save(temp_file.name)
        return temp_file.name
    except Exception:
        print("Thumbnail download failed; using default.")
        img = PILImage.open(BytesIO(requests.get(default_thumbnail_path).content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.thumbnail((800, 600))
        img.save(temp_file.name)
        return temp_file.name

def remove_failed_service(webmap_json, failed_url):
    # Remove from operationalLayers
    if 'operationalLayers' in webmap_json:
        webmap_json['operationalLayers'] = [
            lyr for lyr in webmap_json['operationalLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    # Remove from baseMapLayers
    if 'baseMap' in webmap_json and 'baseMapLayers' in webmap_json['baseMap']:
        webmap_json['baseMap']['baseMapLayers'] = [
            lyr for lyr in webmap_json['baseMap']['baseMapLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    return webmap_json

# For downloads, use in-memory BytesIO where possible
def create_webmap_thumbnail(webmap_json, default_thumbnail_path):
    url = "https://utility.arcgisonline.com/arcgis/rest/services/Utilities/PrintingTools/GPServer/Export%20Web%20Map%20Task/execute"
    #webmap_json = webmap_item.get_data()
    webmap_json = webmap_json if isinstance(webmap_json, dict) else json.loads(webmap_json)
    webmap_json_copy = deepcopy(webmap_json)
    tried_urls = set()
    max_attempts = 10  # Prevent infinite loops

    # List to capture all print service responses
    print_service_response = []

    # Ensure exportOptions is set
    if 'exportOptions' not in webmap_json_copy:
        webmap_json_copy['exportOptions'] = {
            "outputSize": [800, 600],
            "dpi": 96
        }
    # Ensure mapOptions/extent is set
    if 'mapOptions' not in webmap_json_copy:
        webmap_json_copy['mapOptions'] = {}
    if 'extent' not in webmap_json_copy['mapOptions']:
        webmap_json_copy['mapOptions']['extent'] = webmap_json.get('mapOptions', {}).get('extent', webmap_json.get('initialState', {}).get('viewpoint', {}).get('targetGeometry'))

    for attempt in range(max_attempts):
        params = {
            "f": "json",
            "Web_Map_as_JSON": json.dumps(webmap_json_copy),
            "Format": "PNG32",
            "Layout_Template": "MAP_ONLY"
        }
        
        # Capture the final json sent to the print service for troubleshoorting
        # final_webmap_json = deepcopy(webmap_json_copy)
        
        response = requests.post(url, data=params)
        result = response.json()

        # Capture the print service response for troubleshooting
        print_service_response.append({
            "attempt": attempt + 1,
            "params": params,
            "status_code": response.status_code,
            "result": result
        })

        if 'results' in result:
            image_url = result['results'][0]['value']['url']
            img_response = requests.get(image_url)
            if img_response.status_code == 200:
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
                temp_file.write(img_response.content)
                temp_file.close()
                img = PILImage.open(temp_file.name)
                is_blank = is_blank_image(temp_file.name)
                if is_blank:
                    print("Generated thumbnail is blank") #; scaling extent and retrying.")
                    # Try to scale the extent if possible
                    extent = webmap_json_copy.get('mapOptions', {}).get('extent')
                    # if extent:
                    #     new_extent = scale_extent(extent, scale_factor=1.1)
                    #     webmap_json_copy['mapOptions']['extent'] = new_extent
                    #     webmap_json_copy['extent'] = new_extent
                    #     continue  # Retry with new extent
                    # else:
                    if not extent:
                        print("No extent found to scale; using default image.")
                        temp_file.name = create_image_thumbnail(image_url=default_thumbnail_path, default_thumbnail_path=default_thumbnail_path)
                        return temp_file.name, print_service_response, webmap_json
                return temp_file.name, print_service_response, webmap_json
            else:
                break  # No valid image, break and use default

        elif 'error' in result and 'details' in result['error']:
            # Try to extract the failed service URL
            failed_layer_detail = result['error']['details'][0]
            if ' at ' in failed_layer_detail:
                failed_service_url = failed_layer_detail.split(' at ')[-1]
                if failed_service_url in tried_urls:
                    break  # Prevent infinite loop if same URL keeps failing
                tried_urls.add(failed_service_url)
                webmap_json_copy = remove_failed_service(webmap_json_copy, failed_service_url)
                continue  # Try again with the updated JSON
            else:
                break  # Can't parse the failed URL, break and use default
        else:
            break  # No results and no error details, break and use default

    # If we reach here, fallback to default
    print("Thumbnail download failed; using default.")
    temp_file.name = create_image_thumbnail(image_url=default_thumbnail_path, default_thumbnail_path=default_thumbnail_path)
    return temp_file.name, print_service_response, webmap_json

def is_blank_image(image_path, threshold=5):
    img = PILImage.open(image_path).convert('L')
    pixels = list(img.getdata())
    unique_values = set(pixels)
    # If only 1 or 2 unique values (e.g., all black, all white, or half black/half white), treat as blank
    if len(unique_values) <= 2:
        return True
    stat = ImageStat.Stat(img)
    return stat.stddev[0] < threshold  # fallback for nearly-uniform images

# def scale_extent(extent, scale_factor=1.1):
#     """
#     Scales the extent by the given scale_factor (e.g., 1.1 for 10% larger).
#     Extent should be a dict with xmin, ymin, xmax, ymax keys.
#     """
#     if not extent:
#         return extent
#     xmin, ymin, xmax, ymax = extent['xmin'], extent['ymin'], extent['xmax'], extent['ymax']
#     x_center = (xmin + xmax) / 2
#     y_center = (ymin + ymax) / 2
#     width = (xmax - xmin) * scale_factor
#     height = (ymax - ymin) * scale_factor
#     new_xmin = x_center - width / 2
#     new_xmax = x_center + width / 2
#     new_ymin = y_center - height / 2
#     new_ymax = y_center + height / 2
#     return {'xmin': new_xmin, 'ymin': new_ymin, 'xmax': new_xmax, 'ymax': new_ymax, 'spatialReference': extent.get('spatialReference', {'wkid': 102100})}


def build_and_save_storymap(entry, entry_index, entry_title, main_stage_content, new_theme, default_thumbnail_path, gis):
    """
    Build and save a StoryMap from a classic storymap entry.
    Returns the StoryMap object, published item, thumbnail path, print service responses, and webmap JSON.
    """
    print_service_responses = None
    webmap_id_from_entry = None
    webmap_json = None
    media_info = entry.get("media", {})
    media_type = media_info.get("type")
    main_stage_content = main_stage_content
    story = StoryMap()
    story.theme(new_theme)
    sidecar = Sidecar(style="docked-panel")
    story.add(sidecar)

    description_html = entry.get("description", "")
    # Parse HTML and convert to StoryMap nodes
    content_nodes, content_image_metadata = convert_html_elements_to_storymap_node(parse_root_elements(description_html))
    # Add main stage content and text content to sidecar
    sidecar.add_slide(contents=content_nodes, media=main_stage_content)

    # Assign metadata to each image in Side Panel contents
    for img, caption, alt, link in content_image_metadata:
        try:
            img.caption = caption
            img.alt_text = alt
            img.link = link
        except Exception as e:
            print(f"Error setting image metadata: {e}")

    # Set media properties
    if isinstance(main_stage_content, Map):
        # Set webmap properties. Map must be added to the story before setting viewpoint
        if media_type == "webmap":
            # Set the extent for the map stage
            extent_json = media_info.get('webmap', {}).get('extent')
            if extent_json:
                main_stage_content.set_viewpoint(extent=extent_json)  # Extent dict per docs
            # Set layer visibility 
            old_layers = media_info.get('webmap', {}).get('layers', [])
            if old_layers and hasattr(main_stage_content, "map_layers"):
                for new_lyr in main_stage_content.map_layers:
                    for old_lyr in old_layers:
                        if new_lyr['id'] == old_lyr['id']:
                            new_lyr['visible'] = old_lyr['visibility']
            elif "operationalLayers" in media_info.get('webmap', {}):
                old_layers = media_info.get('webmap', {}).get('operationalLayers', [])
                if hasattr(main_stage_content, "map_layers"):
                    for new_lyr in main_stage_content.map_layers:
                        for old_lyr in old_layers:
                            if 'id' in new_lyr and 'id' in old_lyr and new_lyr['id'] == old_lyr['id']:
                                new_lyr['visible'] = old_lyr['visibility']

        # Build a webmap from JSON to create thumbnail
        webmap_id_from_entry, webmap_json = build_webmap_from_json(gis, media_info)
        thumbnail_path, print_service_response, webmap_json = create_webmap_thumbnail(webmap_json=webmap_json, default_thumbnail_path=default_thumbnail_path)
    elif isinstance(main_stage_content, Image):
        image_url = media_info.get("image", {}).get("url")
        thumbnail_path = create_image_thumbnail(image_url=image_url, default_thumbnail_path=default_thumbnail_path)
    # Create a default thumbnail for any unrecongized types
    else:
        thumbnail_path = create_image_thumbnail(image_url=default_thumbnail_path, default_thumbnail_path=default_thumbnail_path)
    # Assign metadata to main stage Images
    if isinstance(main_stage_content, Image):
        if caption:
            main_stage_content.caption = media_info.get("image", {}).get("caption", "")
        if alt:
            main_stage_content.alt_text = media_info.get("image", {}).get("alt", "")
        if link:
            main_stage_content.link = media_info.get("image", {}).get("link", "")
        # if display: # https://developers.arcgis.com/python/latest/api-reference/arcgis.apps.storymap.html#arcgis.apps.storymap.story_content.Image.display
        #    main_stage_content.display = display
        # if properties:
        #    main_stage_content.properties = properties

    # Set cover properties
    cover_properties = story.content_list[0]
    cover_properties.title = entry_title
    cover_properties.byline = ""
    cover_properties.date = "none"
    if not thumbnail_path or not os.path.isfile(thumbnail_path):
        thumbnail_path = default_thumbnail_path
    cover_properties.media = Image(thumbnail_path) 

    # Hide cover. Since the StoryMap cover has no property to hide it, we hide the node using JSON properties
    for k, v in story.properties['nodes'].items():
        if v['type'] == 'storycover':
            v['config'] = {'isHidden': 'true'}

    # Save and publish
    story.save(title=entry_title, tags=["Classic Story Map to AGSM Conversion", "Story Map Series"], publish=True)
    if hasattr(story, '_item'):
        published_story_item = story._item
        published_story_item.update(thumbnail=thumbnail_path)
        published_story_item_url = "https://storymaps.arcgis.com/stories/" + published_story_item.id
        print(f"{published_story_item_url} '{entry_title}' is staged for publishing. Click the link to complete.")
        return story, published_story_item, thumbnail_path
    else:
        print("Could not find item for story:", story.title)
        return story, None, thumbnail_path

def build_collection(classic_item, published_storymap_items, thumbnail_paths, classic_story_type, new_theme):
    collection = Collection()
    collection_title = classic_item.title
    for i, story in enumerate(published_storymap_items):
        if story is None:
            print(f"Story {i+1} is None. Skipping.")
            continue
        try:
            item = Item(gis=gis, itemid=story.itemid)
            resources = item.resources.list()
            published_time = None
            draft_times = []
            for resource in resources:
                if resource['resource'].endswith('published_data.json'):
                    published_time = resource.get('modified')
                elif resource['resource'].startswith('draft_') and resource['resource'].endswith('.json'):
                    modified_time = resource.get('modified')
                    draft_times.append(modified_time)
            if draft_times and published_time and max(draft_times) > published_time:
                print(f"WARNING: There is an issue with '{story.title}'. Click the link to open the story builder and check for errors --> https://storymaps.arcgis.com/stories/{story.itemid}/edit")
            if item.get_data():
                collection.add(item=story, title=story.title, thumbnail=thumbnail_paths[i])
            else:
                print(f"There was a problem publishing '{story.title}'. Open the link {story.url} and try again.")
        except Exception as e:
            print(f"Error adding story to collection: {e}")
    # Set collection properties
    collection.content[0].title = collection_title
    collection.content[0].byline = ""
    collection.theme(new_theme)
    if classic_story_type == "accordion":
        collection.content[1].type = "tab"
    else:
        collection.content[1].type = classic_story_type
    # Set the Collection thumbnail to be the same as the classic story
    classic_thumbnail_path = download_thumbnail(Item(gis=gis, itemid=classic_item.itemid), default_thumbnail_path, gis)
    collection.content[1].media = Image(path=classic_thumbnail_path)
    published_collection = collection.save(title=collection_title, tags=["Classic Story Map to AGSM Conversion", "Story Map Series"], publish=True)
    return collection_title, collection._url, published_collection.id

######################################################

# Instead of using threading.Thread and stopevent, update progress directly after each major step
# def update_progress(progressbar, value, description=''):
#     progressbar.value = value
#     progressbar.description = description if description else progressbar.description

def color_to_hex(color_value):
    """
    Convert a color value (hex, rgb, or named color) to a hex string without the leading '#'.
    """
    color_value = color_value.strip()
    # Check for rgb() format
    rgb_match = re.match(r'rgb-?(\d+)-?(\d+)-?(\d+)', color_value, re.IGNORECASE)
    if rgb_match:
        r, g, b = map(int, rgb_match.groups())
        return '{:02X}{:02X}{:02X}'.format(r, g, b)
    # Check for named color
    try:
        return mcolors.CSS4_COLORS[color_value.lower()].upper()
    except ValueError:
        pass
    # Already hex
    if color_value.startswith('#') and len(color_value) == 7:
        return color_value.upper()
    return None

def convert_color_style_to_class(tag):
    """
    Convert inline color styles to class names and remove inline styles.
    """
    # Check if tag has 'style' attribute with color
    style = tag.get('style', '')
    # Regex to find color property (hex, rgb, named colors)
    match = re.search(r'color\s*:\s*([^;]+)', style, re.IGNORECASE)
    if match:
        color_value = match.group(1).strip()
        # Convert hex (#XXXXXX) to class name, removing #
        if color_value.startswith('#'):
            class_color = f"sm-text-color-{color_value[1:].upper()}"
        else:
            # For rgb or named color, sanitize usable string (replace spaces/paren)
            sanitized = re.sub(r'[\s\(\)]', '', color_value).replace(',', '-')
            hex_color = color_to_hex(sanitized)
            class_color = f"sm-text-color-{hex_color}"
        # Remove color from style attribute
        new_style = re.sub(r'color\s*:\s*[^;]+;?', '', style, flags=re.IGNORECASE).strip()
        if new_style:
            tag['style'] = new_style
        else:
            del tag['style']
        # Add or append class attribute
        if 'class' in tag.attrs:
            tag['class'].append(class_color)
        else:
            tag['class'] = [class_color]

def process_html_colors_preserve_html(html_text):
    """
    Convert inline color styles in HTML text to class names while preserving other HTML tags.   
    """
    soup = BeautifulSoup(html_text, "html.parser")
    # Iterate over tags that can have styles: div, span, strong, em, p, etc.
    for tag in soup.find_all(True):
        convert_color_style_to_class(tag)
    return str(soup)

def convert_element_to_storymap_object(el):
    """
    Convert a BeautifulSoup element to a StoryMap object.
    Returns a tuple of (StoryMap object, caption, alt text, link).
    """
    img_tag = el.find('img')
    if img_tag:
        src = img_tag.get("src")
        # Upgrade http to https if needed
        if src and src.startswith("http://"):
            src = "https://" + src[len("http://"):]
        alt = img_tag.get("alt", "")
        link = "" # TO DO handle occasions when image is intended to launch a link
        # Find figcaption in parent figure or div
        figcaption = ""
        # print("img_tag:", img_tag)
        parent_figure = img_tag.find_parent("figure")
        # print("parent_figure:", parent_figure)
        if parent_figure:
            caption_tag = parent_figure.find("figcaption")
            # print("caption_tag:", caption_tag)
            if caption_tag:
                figcaption = caption_tag.get_text(strip=True)
        else:
            # Try to find figcaption in the parent div
            parent_div = img_tag.find_parent("div")
            # print("parent_div:", parent_div)
            if parent_div:
                caption_tag = parent_div.find("figcaption")
                # print("caption_tag (div):", caption_tag)
                if caption_tag:
                    figcaption = caption_tag.get_text(strip=True)
        # print("Extracted figcaption:", figcaption, type(figcaption))
        img = Image(path=src)
        #img.link = link
        #img.image = src
        return img, figcaption, alt, link

    tag_name = el.name
    if tag_name == "p": # or tag_name in ["span", "strong", "em", "div"]:
        # Extract inner HTML preserving inline styles
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)

    elif tag_name == "video":
        src = el.get("src")
        alt = el.get("alt", "")
        vid = Video(path=src)
        vid.alt_text = alt
        vid.caption = "" # TO DO try to find Classic stories that have Videos with captions
        vid.video = src # Assign video property. TO DO fix this for hosted videos
        return vid
    
    elif tag_name == "audio":
        src = el.get("src")
        alt = el.get("alt", "")
        aud = Audio(path=src)
        aud.alt_text = alt
        aud.caption = "" # TO DO try to find Classic stories that have Audio with captions
        aud.audio = src # Assign Audio property. TO DO fix this for hosted videos
        return aud
    
    elif tag_name == "iframe" or tag_name == "embed":
        src = el.get("src") or el.get("data-src")
        alt = el.get("alt", "")
        if src:
            emb = Embed(path=src)
            emb.alt_text = alt
            emb.caption = "" # TO DO try to find Classic stories that have Embeds with captions
            emb.link = src
        return emb

    elif tag_name == "map":
        src = el.get("src")
        alt = el.get("alt", "")
        extent = "" #TO DO get extent
        layers = "" # TO DO get map layers
        mp = Map(item="")
        mp.alt_text = alt
        mp.caption = "" # TO DO try to find Classic stories that have Maps in Sidecar panel with captions
        mp.map = src
        mp.map_layers = layers 
        mp.set_viewpoint = extent
        return aud
    
    else:
        # Fallback for unsupported or unknown types - treat as text
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)

def parse_root_elements(html_snippet):
    """
    Parse an HTML snippet with BeautifulSoup and return a list of meaningful root-level elements.
    Meaningful elements include those with text, images, videos, audio, iframes, and embeds.
    """
    soup = BeautifulSoup(html_snippet, "html.parser")
    html_elements = []
    for child in soup.contents:
        if not getattr(child, 'name', None):
            continue

        # If this is a <figure> with an <img>, add the whole figure
        if child.name == "figure" and child.find('img'):
            html_elements.append(child)
            continue

        # Check if the parent itself is meaningful
        has_text = child.get_text(strip=True) != ""
        has_img = child.find('img') is not None
        has_video = child.find('video') is not None
        has_audio = child.find('audio') is not None
        has_iframe = child.find('iframe') is not None
        has_embed = child.find('embed') is not None
        has_map = child.find('map') is not None
        is_meaningful = has_text or has_img or has_video or has_audio or has_iframe or has_embed or has_map

        # Check for meaningful children
        meaningful_children = []
        for c in child.children:
            if not getattr(c, 'name', None):
                continue
            c_has_text = c.get_text(strip=True) != ""
            c_has_img = c.find('img') is not None
            c_has_video = c.find('video') is not None
            c_has_audio = c.find('audio') is not None
            c_has_iframe = c.find('iframe') is not None
            c_has_embed = c.find('embed') is not None
            c_has_map = c.find('map') is not None
            if c_has_text or c_has_img or c_has_video or c_has_audio or c_has_iframe or c_has_embed or c_has_map:
                meaningful_children.append(c)

        # If there are meaningful children, add them
        if meaningful_children:
            html_elements.extend(meaningful_children)
            # Optionally, if the parent is also meaningful and not just a container, add it too
            # If you want to avoid duplicates, only add children
            continue

        # If no meaningful children, but parent is meaningful, add parent
        if is_meaningful:
            html_elements.append(child)

    return html_elements


# def parse_nested_elements(html_snippet):
#     soup = BeautifulSoup(html_snippet, "html.parser")
#     soup_list = [child for child in soup.contents if getattr(child, 'name', None)]
#     html_elements = []
#     for element in soup_list:
#         for c in element:
#             if getattr(c, 'name', None):
#                 html_elements.append(c)
#     return html_elements

def convert_html_elements_to_storymap_node(html_elements):
    content_nodes = []
    image_metadata = []  # To store (img, caption, alt, link) tuples
    for el in html_elements:
        node = convert_element_to_storymap_object(el)
        if isinstance(node, tuple):
            img, caption, alt, link = node
            content_nodes.append(img)
            image_metadata.append((img, caption, alt, link))
        elif node:
            content_nodes.append(node)
    return content_nodes, image_metadata

# def get_thumbnail_path(path):
#     try:
#         if path and isinstance(path, str):
#             # Try local file check, otherwise use default
#             with open(path, 'rb'):
#                 return path
#     except Exception:
#         pass
#     # Use default thumbnail path hosted online
#     return default_thumbnail_path
    
# For downloads, use in-memory BytesIO where possible
def download_thumbnail(webmap_item, default_thumbnail_path, gis=gis):
    """
    Download thumbnail from an ArcGIS Online Item to a local temp file and return the local path.
    If download fails, use the default thumbnail path.
    """
    try:
        url = f"{webmap_item._portal.resturl}content/items/{webmap_item.id}/info/{webmap_item.thumbnail}"
        token = gis._con.token if gis else None
        params = {'token': token} if token else {}
        response = requests.get(url, params=params)
        img = PILImage.open(BytesIO(response.content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.save(temp_file.name)
        return temp_file.name
    except Exception:
        print("Thumbnail download failed; using default.")
        url = default_thumbnail_path
        response = requests.get(url)
        img = PILImage.open(BytesIO(response.content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.save(temp_file.name)
        return temp_file.name

#def ensure_local_thumbnail(thumbnail_path, local_filename="default_storymap_thumbnail.png"):
#    # If it's already a local file, just return it
#    if thumbnail_path: # and os.path.isfile(thumbnail_path):
#        return thumbnail_path
#    # Otherwise, download it
#    response = requests.get(thumbnail_path)
#    with open(local_filename, "wb") as f:
#        f.write(response.content)
#    return local_filename

#def create_image_thumbnail(image_url, thumbnail_path):
#    response = requests.get(image_url)
#    img = PILImage.open(BytesIO(response.content))
#    img.thumbnail((800, 600))
#    img.save(thumbnail_path)
#    return thumbnail_path

# Create and displaywidgets
input_param1 = widgets.Checkbox(value=True, description="Yes/No")
user_button1 = widgets.Button(description="Setup Notebook")
user_line1 = widgets.HBox([widgets.Label(value="Are you running this Notebook within ArcGIS Online?"), input_param1])
display(user_line1)
display(widgets.HBox([user_button1]))
user_button1.on_click(setup_notebook)
display(output1)

Initializing...


Output()

## 2. Fetch the Data from the Classic Story Map
This cell fetches the classic StoryMap item and parses its JSON data.

In [ ]:
# Cell 2: Input the classic StoryMap ID
output2 = widgets.Output()
def fetch_story_data(button):
    global classic_item, classic_item_data
    with output2:
        output2.clear_output()
        print("Fetching data...")
        classic_storymap_id = input_param2.value  # or set manually
        classic_item, classic_item_data = fetch_classic_storymap_data(classic_storymap_id, gis)
        if classic_item is None or classic_item_data is not None:
            print(f"Fetched classic StoryMap: '{classic_item.title}' (ID: {classic_item.itemid})")
        else:
            print("Could not fetch classic StoryMap data. Check the item ID and try again.")
        print("\nStep #2 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

input_param2 = widgets.Text(value="d1799fc84e244c2f9af0e24ced4c95e1", description="Item ID:", layout=widgets.Layout(width='400px')) # test value: 597d573e58514bdbbeb53ba2179d2359
user_line2 = widgets.HBox([widgets.Label(value="Paste 32-digit Classic Esri Story Map id -->"), input_param2]) # TO DO add error checking logic and warning if item is missing or input is incorrect
display(user_line2)
user_button2 = widgets.Button(description="Fetch Story Data")
display(user_button2)
display(output2)
user_button2.on_click(fetch_story_data)

Button(description='Fetch Story Data', style=ButtonStyle())

Output()

## 3. Parse Settings, Theme and Data

This cell extracts the theme, title, subtitle, and entries from the classic StoryMap data.

In [ ]:
# Cell 3: Extract settings and entries
output3 = widgets.Output()
def extract_and_display_settings(button):
    global classic_story_title, classic_story_subtitle, classic_story_type, classic_story_panel_position, classic_story_theme, new_theme,entries
    with output3:
        output3.clear_output()
        if classic_item_data is None:
            print("No classic StoryMap data found. Fetch the data first.")
            return
        classic_story_title, classic_story_subtitle, classic_story_type, classic_story_panel_position, classic_story_theme, entries = extract_story_settings(classic_item_data)
        if len(entries) == 1:
            print("\nStory settings:")
            print(f"{'panel position:':>15} {classic_story_panel_position}")
            print(f"{'series title:':>15} '{classic_story_title}'")
            if classic_story_subtitle:
                print(f"{'subtitle:':>15} {classic_story_subtitle}")
            print(f"{'series type:':>15} {classic_story_type}")
            print(f"\nFound {len(entries)} entry in the Classic Map Series.")
        else:
            print("\nStory settings:")
            print(f"{'panel position:':>15} {classic_story_panel_position}")
            print(f"{'series title:':>15} '{classic_story_title}'")
            if classic_story_subtitle:
                print(f"{'subtitle:':>15} {classic_story_subtitle}")
            print(f"{'series type:':>15} {classic_story_type}")
            print(f"\nFound {len(entries)} entries in the Classic Map Series.")    
        for i, e in enumerate(entries):
            print(f"{i+1}. {e['title']}")
        classic_name, new_theme = determine_theme(classic_story_theme)
        print(f"\nClassic theme name: {classic_name}")
        print(f"{'New theme set to:':>19} {new_theme.name}")
        print("\nStep #3 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
user_button3 = widgets.Button(description="Extract Settings")
display(user_button3)
display(output3)
user_button3.on_click(extract_and_display_settings)

Button(description='Extract Settings', style=ButtonStyle())

Output()

## 4. Loop Through and Process Each Entry's Data

In [ ]:
# Cell 4: Loop through entries to process media, content and thumbnails
output4 = widgets.Output()
def process_entries(button):
    global entry_titles, main_stage_contents, invalid_webmaps
    with output4:
        output4.clear_output()
        if entries is None or len(entries) == 0:
            print("No entries found. Extract the story settings first.")
            return
        print(f"Processing {len(entries)} entries...")
        process_all_entries(entries)
        print("\nStep #4 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

def process_all_entries(entries):
    global entry_titles, main_stage_contents, invalid_webmaps
    entry_titles = [None] * len(entries)
    main_stage_contents = [None] * len(entries)
    invalid_webmaps = [False] * len(entries)
    for i, entry in enumerate(entries):
        entry_titles[i], main_stage_contents[i], invalid_webmaps[i] = process_entry(entry)
        if invalid_webmaps[i]:
            print(f"WARNING: There is a problem with the webmap in entry [{i+1} of {len(entries)}]: {entry_titles[i]}. Please fix before publishing the new StoryMap.")
        if type(main_stage_contents[i]).__name__ == "Map":
            webmap_id = entries[i].get("media", {}).get('webmap', {}).get('id')
            print(f"[{i+1} of {len(entries)}]: {entry_titles[i]:35} Media type: {type(main_stage_contents[i]).__name__} (id: {webmap_id})")
        elif type(main_stage_contents[i]).__name__ == "Embed":
            embed_url = entries[i].get("media", {}).get('webpage', {}).get('url')
            print(f"[{i+1} of {len(entries)}]: {entry_titles[i]:35} Media type: {type(main_stage_contents[i]).__name__} (link: {embed_url})")
        elif type(main_stage_contents[i]).__name__ == "Image":
            image_name = entries[i].get("media", {}).get('image', {}).get('title')
            print(f"[{i+1} of {len(entries)}]: {entry_titles[i]:35} Media type: {type(main_stage_contents[i]).__name__} (title: {image_name})")
        else:
            print(f"[{i+1} of {len(entries)}]: {entry_titles[i]:35} Media type: {type(main_stage_contents[i]).__name__}")
input_param4 = widgets.Button(description="Process Entries")
display(input_param4)
display(output4)
input_param4.on_click(process_entries)

## 5. Build an ArcGIS StoryMap with a Suppressed Cover Page for Each Entry

In [ ]:
# Cell 5: Loop through each entry and create a StoryMap for each
output5 = widgets.Output()
def create_storymaps(button):
    global published_storymap_items, thumbnail_paths
    with output5:
        output5.clear_output()
        if entries is None or len(entries) == 0:
            print("No entries found. Extract the story settings first.")
            return
        if main_stage_contents is None or len(main_stage_contents) != len(entries):
            print("No processed entry contents found. Process the entries first.")
            return
        print(f"Creating and saving {len(entries)} StoryMaps...")
        create_and_save_storymaps(entries)
        print("\nStep #5 complete. Ensure you have opened each story and checked for errors before continuing. \nOnce all stories have been successfully published, Click the Markdown text below and then click the 'Play' button twice to proceed.")
        
def create_and_save_storymaps(entries):
    global published_storymap_items, thumbnail_paths
    published_storymap_items = [None] * len(entries)
    thumbnail_paths = [None] * len(entries)
    print("\n***NOTICE*** You MUST click each link below to open the story in a browser tab. ***NOTICE***\n***NOTICE*** Check for errors, edit and continue publishing if necessary.       ***NOTICE***\n\nIf you see an error message -- before troubleshooting further -- try just clicking the 'Publish' button. Doing so can fix many common issues.\n")
    for i, entry in enumerate(entries):
        print(f"[{i+1} of {len(entries)}]... ",end="")
        story, published_story_item, thumbnail_path = build_and_save_storymap(entry, i, entry["title"], main_stage_contents[i], new_theme, default_thumbnail_path, gis)
        if published_story_item:
            published_storymap_items[i] = published_story_item
        thumbnail_paths[i] = thumbnail_path
user_button6 = widgets.Button(description="Create ArcGIS StoryMaps from each entry", layout=widgets.Layout(width='300px', height='40px'))    
display(user_button6)
display(output5)
user_button6.on_click(create_storymaps)

## 6. Build a Collection from the Published StoryMaps

In [ ]:
# Cell 6. Run the function to create the Collection
output6 = widgets.Output()
def create_collection(button):
    global collection_title, collection_url, collection_id
    with output6:
        output6.clear_output()
        print(f"Creating Collection '{classic_story_title}'...")
        if published_storymap_items is None or len(published_storymap_items) == 0:
            print("No published StoryMap items found. Create the StoryMaps first.")
            return
        collection_title, collection_url, collection_id = build_collection(classic_item, published_storymap_items, thumbnail_paths, classic_story_type, new_theme)
        print(f"Collection created: '{collection_title}' {collection_url} \nClick the link to open the Collection builder. Make any desired edits and then complete the publication of your converted StoryMap.")
        print("\nStep #6 complete. Once you've published the Collection, click the Markdown text below and then click the 'Play' button twice to proceed.")
user_button7 = widgets.Button(description="Create Collection", layout=widgets.Layout(width='150px', height='40px'))    
display(user_button7)
display(output6)
user_button7.on_click(create_collection)


## 7. Create a folder in My Content to store the results (optional, but recommended) 

In [ ]:
# Cell 7: Create a folder to store the results
output7 = widgets.Output()
input_param7 = widgets.Text(value="", description="", layout=widgets.Layout(width='800px'))
user_button7 = widgets.Button(description="Check for folder")
user_button7_1 = widgets.Button(description="Create folder")

user_button7.on_click(check_folder)
user_button7_1.on_click(create_folder)

display(user_button7)
display(output7)

## 8. Move content into folder for easier management

In [ ]:
# Cell 8. Move all items to the newly created folder
output8 = widgets.Output()
def move_items_to_folder(button):
    global folder_name, classic_item, published_storymap_items, collection_title
    with output8:
        output8.clear_output()
        if folder_name is None:
            print("No folder name found. Check for or create a folder first.")
            return
        if classic_item is None:
            print("No classic StoryMap item found. Fetch the story data first.")
            return
        if published_storymap_items is None or len(published_storymap_items) == 0:
            print("No published StoryMap items found. Create the StoryMaps first.")
            return
        if collection_title is None:
            print("No collection found. Create the collection first.")
            return
        print(f"Moving items to folder '{folder_name}'...")
        move_item_to_folder(gis, classic_item, folder_name)
        for story_item in published_storymap_items:
            if story_item:
                move_item_to_folder(gis, story_item, folder_name)
        # Move the collection item
        try:
            #collection_search = gis.content.search(query=f'title:"{collection_title}" AND owner:{gis.users.me.username}', item_type="Collection", max_items=1)
            if collection_id:
                collection_item = gis.content.get(collection_id)
                move_item_to_folder(gis, collection_item, folder_name)
                print(f"Moved collection '{collection_title}' to folder '{folder_name}'.")
            else:
                print(f"Could not find the collection item '{collection_title}' to move.")
        except Exception as e:
            print(f"Error moving collection item: {e}")
        print("\nStep #8 complete. Conversion complete!")

def move_item_to_folder(gis, item, folder_name):
    try:
        if item.owner == gis.users.me.username:
            item.move(folder_name)
            print(f"Moved item '{item.title}' (ID: {item.itemid}) to folder '{folder_name}'.")
    except Exception as e:
        print(f"Error moving item '{item.title}' (ID: {item.itemid}): {e}")

user_button8 = widgets.Button(description="Move all items to folder", layout=widgets.Layout(width='200px', height='40px'))    
display(user_button8)
display(output8)
user_button8.on_click(move_items_to_folder)